## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [4]:
from os import getcwd, path, pardir

## Compile and program project

In [5]:
project_path_s = path.join(getcwd(), "TrustZone-S")
project_path_s

'C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Experiments\\Security\\TrustZone\\TrustZone-S'

In [6]:
project_path_ns = path.join(getcwd(), "TrustZone-NS")
project_path_ns

'C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Experiments\\Security\\TrustZone\\TrustZone-NS'

In [7]:
atprogram(project_path_ns, verbose=2)
atprogram(project_path_s, verbose=2, erase=False)

Building file: .././main.c
Invoking: ARM/GNU C Compiler : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\bin\arm-none-eabi-gcc.exe"  -x c -mthumb -D__SAML11E16A__ -DDEBUG  -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\arm\cmsis\5.0.1\CMSIS\Include" -I"../Config" -I".." -I"../examples" -I"../hal/include" -I"../hal/utils/include" -I"../hpl/core" -I"../hpl/crya" -I"../hpl/dmac" -I"../hpl/gclk" -I"../hpl/mclk" -I"../hpl/osc32kctrl" -I"../hpl/oscctrl" -I"../hpl/pm" -I"../hpl/port" -I"../hri" -I"../../TrustZone-S/trustzone" -I"C:\Program Files (x86)\Atmel\Studio\7.0\Packs\Atmel\SAML11_DFP\1.0.91\include"  -O1 -ffunction-sections -mlong-calls -g3 -Wall -mcpu=cortex-m23 -c -std=gnu99 -MD -MP -MF "main.d" -MT"main.d" -MT"main.o"   -o "main.o" ".././main.c" 
Finished building: .././main.c
Building target: TrustZone-NS.elf
Invoking: ARM/GNU Linker : 6.3.1
"C:\Program Files (x86)\Atmel\Studio\7.0\toolchain\arm\arm-gnu-toolchain\bin\arm-none-eabi-gcc.exe" -o Trust

0

## Data Logging

In [7]:
live_plot = True

Create a figure for the plot.

In [8]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [9]:
if live_plot:
    config_dict = {
        "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
        "plot_pins": [False, False, True, True],
        "gpio_delay_time" : 0.0007,
        "plot_pins_method": "line",
        "plot_xmax": 1,
        "fig": fig,
        "window_title": "Experiment TrustZone",
        "file_name_base": "experiment_trustzone"
    }
else:
    config_dict = {
        "loggers": [LOGGER_OBJECT, LOGGER_CSV],
        "gpio_delay_time" : 0.0007,
        "file_name_base": "experiment_trustzone"
    }

Stop criteria to pass to the logger:

In [11]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Perform the measurement.

In [13]:
data = []
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()

with DGILibExtra(**config_dict) as dgilib:
    dgilib.device_reset()
    data = dgilib.logger.log(400,stop_fn)
    data = dgilib.data

In [14]:
print(data)

Interfaces:
	 256:   power,   samples: 3551000
	  48:   gpio,    samples:   32644



## Analysis

Create Stop Function to stop parsing the data when all pins are high.

In [15]:
def stop_function(pin_values):
    return all(pin_values)

Parse the data.

In [16]:
nsc_store_charge, nsc_store_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)

In [17]:
nsc_load_charge, nsc_load_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

In [18]:
MIN_NUM_BYTES = 1
num_bytes = range(MIN_NUM_BYTES, MIN_NUM_BYTES + len(nsc_store_charge))
print(f"MAX_NUM_BYTES: {MIN_NUM_BYTES + len(nsc_store_charge) - 1}")

MAX_NUM_BYTES: 4080


In [19]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)

In [20]:
results = []
for y in [nsc_store_charge, nsc_load_charge, nsc_store_time, nsc_load_time]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 4080
    # variables        = 2
    chi-square         = 6.0008e-13
    reduced chi-square = 1.4715e-16
    Akaike info crit   = -148734.738
    Bayesian info crit = -148722.111
[[Variables]]
    slope:      8.6781e-10 +/- 1.6124e-13 (0.02%) (init = 0)
    intercept:  2.2524e-08 +/- 3.7989e-10 (1.69%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.866

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 4080
    # variables        = 2
    chi-square         = 5.7582e-13
    reduced chi-square = 1.4120e-16
    Akaike info crit   = -148903.059
    Bayesian info crit = -148890.431
[[Variables]]
    slope:      8.7152e-10 +/- 1.5795e-13 (0.02%) (init = 0)
    intercept:  1.8287e-08 +/- 3.7214e-10 (2.03%) (init = 1)
[[Correlations]] (unrep

In [21]:
fig2 = plt.figure(figsize=(9, 8))
fig2.canvas.set_window_title("Analysis TrustZone")

In [22]:
charge_color = 'r'
time_color = 'b'

In [23]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Number of bytes')
ax1.set_ylabel('Charge [C]', color=charge_color)
ax2.set_ylabel('Time [s]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [24]:
lines = []
lines += ax1.plot(num_bytes, nsc_store_charge, charge_color+'-', label='TrustZone Store Charge')
lines += ax1.plot(num_bytes, nsc_load_charge, charge_color+'--', label='TrustZone Load Charge')
lines += ax2.plot(num_bytes, nsc_store_time, time_color+'-', label='TrustZone Store Time')
lines += ax2.plot(num_bytes, nsc_load_time, time_color+'--', label='TrustZone Load Time')
ax1.legend(handles=lines)
# [nsc_store_charge, nsc_load_charge, nsc_store_time, nsc_load_time]
ax1.set_title(f"TrustZone Store Charge: Base {results[0].params['intercept'].value:.03} C plus {results[0].params['slope'].value:.03} mC per byte\n" +
             f"TrustZone Load Charge: Base {results[1].params['intercept'].value:.03} C plus {results[1].params['slope'].value:.03} mC per byte\n" +
             f"TrustZone Store Time: Base {results[2].params['intercept'].value:.03} s plus {results[2].params['slope'].value:.03} s per byte\n" +
             f"TrustZone Load Time: Base {results[3].params['intercept'].value:.03} s plus {results[3].params['slope'].value:.03} s per byte\n")
fig2.tight_layout()
fig2.show()

In [25]:
# Save Charge amount list into pickle file
import pickle
pickle.dump(nsc_store_charge, open("trustzone_store_charge.p", "wb"))
pickle.dump(nsc_load_charge, open("trustzone_load_charge.p", "wb"))

# Python script to do measurement

In [26]:
%%writefile experiment_trustzone.py

from os import getcwd, path
import pickle

from pydgilib_extra import *
from atprogram.atprogram import atprogram

def experiment_trustzone(project_root=getcwd(), verbose=1):
    project_path_s = path.join(project_root, "TrustZone-S")
    project_path_ns = path.join(project_root, "TrustZone-NS")
    atprogram(project_path_ns, verbose=verbose)
    atprogram(project_path_s, verbose=verbose, erase=False)

    config_dict = {
        "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
        "loggers": [LOGGER_OBJECT, LOGGER_CSV],
        "gpio_delay_time" : 0.0007,
        "file_name_base": "experiment_trustzone",
        "log_folder": project_root,
        "verbose": verbose-1
    }
    
    def stop_fn(logger_data):
        return all(logger_data.gpio.values[-1])
    
    if verbose:
        print(f"Start DGILibExtra with config: \n{config_dict}\n")

    data = []
    with DGILibExtra(**config_dict) as dgilib:
        dgilib.device_reset()
        dgilib.logger.log(500,stop_fn)
        data = dgilib.data
        
    if verbose:
        print(f"DGILibExtra data: {data}")

    def stop_function(pin_values):
        return all(pin_values)

    nsc_store_charge, nsc_store_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)
    nsc_load_charge, nsc_load_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

    # Save Charge amount list into pickle file
    pickle.dump(nsc_store_charge, open(path.join(project_root, "trustzone_store_charge.p"), "wb"))
    pickle.dump(nsc_load_charge, open(path.join(project_root, "trustzone_load_charge.p"), "wb"))
    
    if verbose:
        print(f"Saved results in: {path.join(project_root)}")

if __name__ == "__main__":
   experiment_trustzone()

Overwriting experiment_trustzone.py
